In [1]:
from bert_serving.client import BertClient
import numpy as np
import pandas as pd

In [2]:
from tqdm import tqdm
from preprocess import read_articles_from_file_list, read_predictions_from_file, label2index
bc = BertClient(ip='10.2.0.111')

Using TensorFlow backend.


In [3]:
a = bc.encode(['First do it', 'then do it right', 'then do it better'])

In [4]:
train_folder = "datasets/train-articles"  # check that the path to the datasets folder is correct,
dev_folder = "datasets/dev-articles"  # if not adjust these variables accordingly
train_labels_file = "datasets/train-task2-TC.labels"
dev_template_labels_file = "datasets/dev-task-TC-template.out"
task_TC_output_file = "baseline-output-TC.txt"

In [5]:
def clean_text(text):
    # text = text.lower()
    # text = text.replace('\'', '')
    text = text.replace('‘', ' \' ')
    text = text.replace('’', ' \' ')
    text = text.replace('“', ' \" ')
    text = text.replace('”', ' \" ')
    text = text.replace('â', ' \' ')

    text = text.replace('"', ' " ')
    text = text.replace('\'', ' \' ')

    text = text.replace('—', ' - ')
    text = text.replace('–', ' - ')
    text = text.replace('…', '...')
    text = text.replace('  ', ' ')
    text = text.strip()
    return text

In [6]:
articles = read_articles_from_file_list(train_folder)
dev_articles = read_articles_from_file_list(dev_folder)
ref_articles_id, ref_span_starts, ref_span_ends, train_gold_labels = read_predictions_from_file(train_labels_file)
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)
print("Loaded %d annotations from %d articles" % (len(ref_span_starts), len(set(ref_articles_id))))


Loaded 6129 annotations from 357 articles


In [7]:
def compute_features(articles, ref_articles_id, span_starts, span_ends):
    # only one feature, the length of the span
    print(type(span_starts), len(span_starts))
    print(type(span_ends), len(span_ends))
    data = []
    article_spans = []
    for i, ref_id in tqdm(enumerate(ref_articles_id)):
        # print(articles[ref_id], span_starts[i], span_ends[i])
        article = articles[ref_id]
        article_span = clean_text(article[int(span_starts[i]):int(span_ends[i])])
        data.append([article_span])
        article_spans.append(article_span)
        
    return article_spans

In [8]:
articles = compute_features(articles, ref_articles_id, ref_span_starts, ref_span_ends)
dev_articles = compute_features(dev_articles, dev_article_ids, dev_span_starts, dev_span_ends)

6129it [00:00, 464400.49it/s]
1063it [00:00, 233505.04it/s]

<class 'list'> 6129
<class 'list'> 6129
<class 'list'> 1063
<class 'list'> 1063


In [9]:
from keras.utils import to_categorical
from pprint import pprint
import pickle
from preprocess import label2index, index2label

In [10]:
GET_NEW_BERT_EMB = True

In [25]:
data_path = './bert_processed_data/'
if GET_NEW_BERT_EMB:
    articles_emb = bc.encode(articles)
    dev_articles_emb = bc.encode(dev_articles)
    pprint(set(train_gold_labels))

    labels = [label2index[x] for x in train_gold_labels]
    labels = to_categorical(np.asarray(labels))

    # save train data
    pickle.dump(articles_emb, open(data_path + 'train_x.p', 'wb'))
    pickle.dump(labels, open(data_path + 'train_y.p', 'wb'))

    # save dev data
    pickle.dump(dev_articles_emb, open(data_path + 'dev_x.p', 'wb'))
    
else:
    articles_emb = pickle.load(open(data_path + 'train_x.p', 'rb'))
    labels = pickle.load(open(data_path + 'train_y.p', 'rb'))
    dev_articles_emb = pickle.load(open(data_path + 'dev_x.p', 'rb'))

/home/gangeshwark/anaconda3/lib/python3.7/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=64" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


{'Appeal_to_Authority',
 'Appeal_to_fear-prejudice',
 'Bandwagon,Reductio_ad_hitlerum',
 'Black-and-White_Fallacy',
 'Causal_Oversimplification',
 'Doubt',
 'Exaggeration,Minimisation',
 'Flag-Waving',
 'Loaded_Language',
 'Name_Calling,Labeling',
 'Repetition',
 'Slogans',
 'Thought-terminating_Cliches',
 'Whataboutism,Straw_Men,Red_Herring'}


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
train_x, test_x, train_y, test_y = train_test_split(articles_emb,
                                                  labels,
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  stratify=labels)

In [28]:
from keras.layers import Conv2D, MaxPool2D, Bidirectional
from keras.layers import Embedding, Concatenate
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

def model_MLP(emb_size, out_size):
    model = Sequential()
    model.add(Dense(128, input_dim=emb_size, activation='relu'))
    model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(out_size, activation='softmax'))
    print(model.summary())
    return model

In [29]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from models import load_model
dev_template_labels_file = "datasets/dev-task-TC-template.out"


emb_size = articles_emb[0].shape[0]
print(emb_size)
out_size = 14

model = model_MLP(emb_size, out_size)

lr = 0.0001
bz = 256
epochs = 300

opt = Adam(lr=lr)
# opt = SGD(0.01)
# print(str(opt))
# exit()
model_name = 'bert_text_Adam_lr%s_bz%s' % (lr, bz)
model_path = 'models/%s' % (model_name)
checkpoint = ModelCheckpoint('%s.{epoch:02d}.hdf5' % (model_path), monitor='loss', verbose=1,
                             save_best_only=False, mode='auto')

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])
try:
    model.fit(train_x, train_y, validation_data=[test_x, test_y], epochs=epochs,
              batch_size=bz,
              shuffle=True, callbacks=[checkpoint])
except:
    pass


12288
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               1572992   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 14)                1806      
Total params: 1,574,798
Trainable params: 1,574,798
Non-trainable params: 0
_________________________________________________________________
None
Train on 4903 samples, validate on 1226 samples
Epoch 1/300
4903/4903 [==============================] - 1s 103us/step - loss: 2.3602 - acc: 0.3121 - val_loss: 1.8236 - val_acc: 0.4119

Epoch 00001: saving model to models/bert_text_Adam_lr0.0001_bz256.01.hdf5
Epoch 2/300
4903/4903 [==============================] - 0s 32us/step - loss: 1.8821 - acc: 0.4071 - va

Epoch 36/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.8767 - acc: 0.7200 - val_loss: 1.0734 - val_acc: 0.6599

Epoch 00036: saving model to models/bert_text_Adam_lr0.0001_bz256.36.hdf5
Epoch 37/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.8736 - acc: 0.7216 - val_loss: 1.0774 - val_acc: 0.6533

Epoch 00037: saving model to models/bert_text_Adam_lr0.0001_bz256.37.hdf5
Epoch 38/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.8726 - acc: 0.7251 - val_loss: 1.0822 - val_acc: 0.6542

Epoch 00038: saving model to models/bert_text_Adam_lr0.0001_bz256.38.hdf5
Epoch 39/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.8664 - acc: 0.7263 - val_loss: 1.0648 - val_acc: 0.6550

Epoch 00039: saving model to models/bert_text_Adam_lr0.0001_bz256.39.hdf5
Epoch 40/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.8356 - acc: 0.7359 - val_loss: 1.0683 - val_acc: 0.6607

Epoch 00040: s


Epoch 00074: saving model to models/bert_text_Adam_lr0.0001_bz256.74.hdf5
Epoch 75/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.5754 - acc: 0.8179 - val_loss: 1.0513 - val_acc: 0.6721

Epoch 00075: saving model to models/bert_text_Adam_lr0.0001_bz256.75.hdf5
Epoch 76/300
4903/4903 [==============================] - 0s 31us/step - loss: 0.5596 - acc: 0.8262 - val_loss: 1.0529 - val_acc: 0.6648

Epoch 00076: saving model to models/bert_text_Adam_lr0.0001_bz256.76.hdf5
Epoch 77/300
4903/4903 [==============================] - 0s 28us/step - loss: 0.5589 - acc: 0.8224 - val_loss: 1.0605 - val_acc: 0.6648

Epoch 00077: saving model to models/bert_text_Adam_lr0.0001_bz256.77.hdf5
Epoch 78/300
4903/4903 [==============================] - 0s 32us/step - loss: 0.5572 - acc: 0.8242 - val_loss: 1.0502 - val_acc: 0.6729

Epoch 00078: saving model to models/bert_text_Adam_lr0.0001_bz256.78.hdf5
Epoch 79/300
4903/4903 [==============================] - 0s 39us/step - loss

4903/4903 [==============================] - 0s 29us/step - loss: 0.3968 - acc: 0.8805 - val_loss: 1.0819 - val_acc: 0.6688

Epoch 00113: saving model to models/bert_text_Adam_lr0.0001_bz256.113.hdf5
Epoch 114/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.3882 - acc: 0.8846 - val_loss: 1.1167 - val_acc: 0.6615

Epoch 00114: saving model to models/bert_text_Adam_lr0.0001_bz256.114.hdf5
Epoch 115/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.3814 - acc: 0.8870 - val_loss: 1.1031 - val_acc: 0.6688

Epoch 00115: saving model to models/bert_text_Adam_lr0.0001_bz256.115.hdf5
Epoch 116/300
4903/4903 [==============================] - 0s 45us/step - loss: 0.3717 - acc: 0.8927 - val_loss: 1.1092 - val_acc: 0.6713

Epoch 00116: saving model to models/bert_text_Adam_lr0.0001_bz256.116.hdf5
Epoch 117/300
4903/4903 [==============================] - 0s 32us/step - loss: 0.3862 - acc: 0.8911 - val_loss: 1.1273 - val_acc: 0.6680

Epoch 00117: saving

4903/4903 [==============================] - 0s 29us/step - loss: 0.2981 - acc: 0.9125 - val_loss: 1.1485 - val_acc: 0.6672

Epoch 00151: saving model to models/bert_text_Adam_lr0.0001_bz256.151.hdf5
Epoch 152/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.2969 - acc: 0.9113 - val_loss: 1.1757 - val_acc: 0.6664

Epoch 00152: saving model to models/bert_text_Adam_lr0.0001_bz256.152.hdf5
Epoch 153/300
4903/4903 [==============================] - 0s 34us/step - loss: 0.2888 - acc: 0.9164 - val_loss: 1.1719 - val_acc: 0.6664

Epoch 00153: saving model to models/bert_text_Adam_lr0.0001_bz256.153.hdf5
Epoch 154/300
4903/4903 [==============================] - 0s 36us/step - loss: 0.2798 - acc: 0.9241 - val_loss: 1.1897 - val_acc: 0.6729

Epoch 00154: saving model to models/bert_text_Adam_lr0.0001_bz256.154.hdf5
Epoch 155/300
4903/4903 [==============================] - 0s 31us/step - loss: 0.2824 - acc: 0.9205 - val_loss: 1.1902 - val_acc: 0.6623

Epoch 00155: saving

4903/4903 [==============================] - 0s 29us/step - loss: 0.2250 - acc: 0.9351 - val_loss: 1.2447 - val_acc: 0.6680

Epoch 00189: saving model to models/bert_text_Adam_lr0.0001_bz256.189.hdf5
Epoch 190/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.2198 - acc: 0.9396 - val_loss: 1.2546 - val_acc: 0.6607

Epoch 00190: saving model to models/bert_text_Adam_lr0.0001_bz256.190.hdf5
Epoch 191/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.2226 - acc: 0.9376 - val_loss: 1.2589 - val_acc: 0.6656

Epoch 00191: saving model to models/bert_text_Adam_lr0.0001_bz256.191.hdf5
Epoch 192/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.2329 - acc: 0.9311 - val_loss: 1.2519 - val_acc: 0.6656

Epoch 00192: saving model to models/bert_text_Adam_lr0.0001_bz256.192.hdf5
Epoch 193/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.2248 - acc: 0.9325 - val_loss: 1.2576 - val_acc: 0.6639

Epoch 00193: saving

4903/4903 [==============================] - 0s 30us/step - loss: 0.1848 - acc: 0.9508 - val_loss: 1.3434 - val_acc: 0.6615

Epoch 00227: saving model to models/bert_text_Adam_lr0.0001_bz256.227.hdf5
Epoch 228/300
4903/4903 [==============================] - 0s 41us/step - loss: 0.1700 - acc: 0.9504 - val_loss: 1.3340 - val_acc: 0.6656

Epoch 00228: saving model to models/bert_text_Adam_lr0.0001_bz256.228.hdf5
Epoch 229/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.1752 - acc: 0.9486 - val_loss: 1.3219 - val_acc: 0.6623

Epoch 00229: saving model to models/bert_text_Adam_lr0.0001_bz256.229.hdf5
Epoch 230/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.1778 - acc: 0.9502 - val_loss: 1.3422 - val_acc: 0.6680

Epoch 00230: saving model to models/bert_text_Adam_lr0.0001_bz256.230.hdf5
Epoch 231/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.1774 - acc: 0.9460 - val_loss: 1.3122 - val_acc: 0.6697

Epoch 00231: saving

4903/4903 [==============================] - 0s 29us/step - loss: 0.1523 - acc: 0.9525 - val_loss: 1.4259 - val_acc: 0.6672

Epoch 00265: saving model to models/bert_text_Adam_lr0.0001_bz256.265.hdf5
Epoch 266/300
4903/4903 [==============================] - 0s 28us/step - loss: 0.1555 - acc: 0.9519 - val_loss: 1.3843 - val_acc: 0.6680

Epoch 00266: saving model to models/bert_text_Adam_lr0.0001_bz256.266.hdf5
Epoch 267/300
4903/4903 [==============================] - 0s 31us/step - loss: 0.1559 - acc: 0.9553 - val_loss: 1.4041 - val_acc: 0.6639

Epoch 00267: saving model to models/bert_text_Adam_lr0.0001_bz256.267.hdf5
Epoch 268/300
4903/4903 [==============================] - 0s 31us/step - loss: 0.1550 - acc: 0.9506 - val_loss: 1.4526 - val_acc: 0.6648

Epoch 00268: saving model to models/bert_text_Adam_lr0.0001_bz256.268.hdf5
Epoch 269/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.1619 - acc: 0.9484 - val_loss: 1.4289 - val_acc: 0.6680

Epoch 00269: saving

In [30]:
epoch = input("\n\nWhich epoch to load?\nAns: ")
epoch = int(epoch)
load_model_path = '%s.%02d.hdf5' % (model_path, epoch)
print('Loading model - ', load_model_path)
model = load_model(load_model_path, custom_objects={'loss': 'categorical_crossentropy'})
# print(model.summary())
predictions = model.predict(dev_articles_emb)
predictions = predictions.argmax(axis=1)

# writing predictions to file
task_TC_output_file = "BERT_model-output-TC.txt"
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)

with open(task_TC_output_file, "w") as fout:
    for article_id, prediction, span_start, span_end in zip(dev_article_ids, predictions, dev_span_starts,
                                                            dev_span_ends):
        fout.write("%s\t%s\t%s\t%s\n" % (article_id, index2label[prediction], span_start, span_end))
print("Predictions written to file " + task_TC_output_file)



Which epoch to load?
Ans: 69
Loading model -  models/bert_text_Adam_lr0.0001_bz256.69.hdf5
Predictions written to file BERT_model-output-TC.txt
